In [1]:
import numpy as np
import time
import os

In [2]:
scenario = "6"
for i in range(100):
    files_name = "hdgp/scenario_" + scenario +"/hdgp_" + scenario + "_" + str(i) + ".npz"
    data = np.load(files_name)
    data.close()

In [3]:
i = 0
files_name = "hdgp/scenario_" + scenario +"/hdgp_" + scenario + "_" + str(i) + ".npz"
data = np.load(files_name)
print(data.files)
print(data['SNP'])
print(data['POS'])

['SNP', 'POS']
[[0 0 1 ... 1 0 0]
 [1 0 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 1 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]
[ 122 1062  201 ...   13  923  429]


In [4]:
def SNPtoSMC(dist, SNP, POS):
    """
    dist : distinguished ind
    SNP : snp matrix
    POS : positions matrix of SNP
    """
    N = SNP.shape[0]
    if(N%2 == 1):
        N -= 1
    M = POS.shape[0]
    SMC = np.zeros((2*M,4), dtype=int)
    SMC[:,-1] = N*np.ones(2*M, dtype=int) - 2
    topop = []
    for i in range(M):
        span = POS[i]
        if(span > 0):
            d = u = 0
            SMC[2*i, :-1] = [span, d, u]
        else:
            topop += [2*i]
        
        span = 1
        d = SNP[dist, i] + SNP[dist+1, i]
        u = SNP[:, i].sum() - d
        SMC[2*i+1, :-1] = [span, d, u]
        
    return np.delete(SMC, topop, 0)

def save(filename, SMP):
    np.savetxt(filename + ".smc", SMP, delimiter=' ')

In [5]:
t0 = time.time()
SMC = SNPtoSMC(0, data['SNP'], data['POS'])
t1 = time.time()
print(f"time: {t1 - t0:.4f}")
print(SMC[:30])
print(sorted(SMC[:,0]))

time: 0.1003
[[ 122    0    0   34]
 [   1    1    3   34]
 [1062    0    0   34]
 [   1    0    5   34]
 [ 201    0    0   34]
 [   1    1    0   34]
 [ 991    0    0   34]
 [   1    0    1   34]
 [ 102    0    0   34]
 [   1    0    4   34]
 [  30    0    0   34]
 [   1    0    5   34]
 [ 180    0    0   34]
 [   1    2   30   34]
 [ 450    0    0   34]
 [   1    2   30   34]
 [ 269    0    0   34]
 [   1    2   30   34]
 [  51    0    0   34]
 [   1    0    5   34]
 [  18    0    0   34]
 [   1    2   30   34]
 [ 163    0    0   34]
 [   1    0    5   34]
 [ 325    0    0   34]
 [   1    0    1   34]
 [ 427    0    0   34]
 [   1    2   30   34]
 [ 338    0    0   34]
 [   1    0    5   34]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [6]:
snp = np.array([
[0, 0, 1, 1],
[0, 0, 0, 1],
[1, 0, 0, 0],
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
])

pos = np.array([3, 5, 0, 10])
SNPtoSMC(0, snp, pos)

array([[ 3,  0,  0,  4],
       [ 1,  0,  2,  4],
       [ 5,  0,  0,  4],
       [ 1,  0,  1,  4],
       [ 1,  1,  0,  4],
       [10,  0,  0,  4],
       [ 1,  2,  1,  4]])

In [7]:
def boucle():
    for d in os.listdir("hdgp"):
        for f in os.listdir("hdgp/" + d):
            data = np.load("hdgp/" + d + "/" + f)
            N = data['SNP'].shape[0]
            if N % 2 == 1:
                N -= 1
            for dist in range(0, N, 2):
                SMC = SNPtoSMC(dist, data['SNP'], data['POS'])
                filename = "out/" + d + "/" + f + str(dist) + ".smc"
                header = 'SMC++ {"version": "1.15.4.dev18+gca077da", "pids": ["hdgp"], "undist": [[]], "dist": [[["HGDP01396", 0], ["HGDP01396", 1]]]}'
                np.savetxt(filename , SMC, delimiter=' ', fmt = "%d", header=header)
        return ## on fait qu'un seul dossier

In [8]:
boucle()